In [61]:
import pandas as pd

In [62]:
df = pd.read_csv('c:\\Users\\Ken\\Documents\\GitHub\\coursera_rag\\requirements_from_acts.csv')
df.fillna('blank', inplace=True)  # remove any NaN values as it blows up serialization
data = df.to_dict('records')
len(data)

227

In [63]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [64]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [65]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [66]:
# Create collection to store records
qdrant.recreate_collection(
    collection_name="legal_reqts",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [67]:
# vectorize
qdrant.upload_points(
    collection_name="legal_reqts",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Essential Requirement"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the records
    ]
)

In [68]:
# Start the llamafile server

import subprocess

# Command to start the llamafile server
server_process = subprocess.Popen(['C:\\Users\\Ken\\Documents\\GitHub\\coursera_rag\\.venv\\llava-v1.5-7b-q4.llamafile.exe'])


In [69]:
# this is structuring the database search
user_prompt = "Identify all requirements for risk analysis"

In [70]:
# Search source

hits = qdrant.search(
    collection_name="legal_reqts",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'ID': 69, 'Document ID': '2006/42/EC', 'Document Title': 'Machinery Directive', 'Annex': 'I', 'Section L1': 1, 'Section L2': 1.0, 'Section L3': 2.0, 'Section L4': '(b)', 'Essential Requirement': '1.1.2.(b) In selecting the most appropriate methods, the manufacturer or his authorised representative must apply the following principles, in the order given: — eliminate or reduce risks as far as possible (inherently safe machinery design and construction), — take the necessary protective measures in relation to risks that cannot be eliminated, — inform users of the residual risks due to any shortcomings of the protective measures adopted, indi\x02cate whether any particular training is required and specify any need to provide personal protective equipment.'} score: 0.5584695383993481
{'ID': 136, 'Document ID': '2006/42/EC', 'Document Title': 'Machinery Directive', 'Annex': 'I', 'Section L1': 1, 'Section L2': 7.0, 'Section L3': 2.0, 'Section L4': 'blank', 'Essential Requirement': '1.7.2. Wa

In [71]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [72]:
# Connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)

completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are a chatbot, skilled in explaining regulatory requirements for machinery."},
        {"role": "user", "content": "Identify requirements for risk analysis as part of CE marking."},
        {"role": "assistant", "content": str(search_results)}
    ]
)

output_message = completion.choices[0].message
import csv

# Data to be written to CSV
data = [
    {"role": "system", "content": "You are a chatbot, skilled in explaining regulatory requirements for machinery."},
    {"role": "user", "content": "Identify requirements for risk analysis as part of CE marking."},
    {"role": output_message.role, "content": output_message.content}
]
# Write data to a CSV file
with open("C:\\Users\\Ken\\Downloads\\output.csv", "w", newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["role", "content"])
    writer.writeheader()
    writer.writerows(data)

print("Output written to file")

Output written to file


In [73]:
# Terminate the llamafile server

import signal

server_process.send_signal(signal.SIGTERM)